In [1]:
from math import log
import operator

def calcShannonEntropy(dataset):
    numEntries = len(dataset)
    
    labelCounts = {}
    
    for featureVector in dataset:
        currentLabel = featureVector[-1]
        
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        
        labelCounts[currentLabel] += 1
        
    shannonEntropy = 0.0
    
    for key in labelCounts:
        probability = float(labelCounts[key]) / numEntries
        shannonEntropy -= probability * log(probability,2)
            
    return shannonEntropy
    

In [2]:
def createDataset():
    dataset = [[1,1,'yes'],[1,1,'yes'],[1,0,'no'],[0,1,'no'],[0,1,'no']]
    
    labels = ['no surfacing', 'flippers']
    
    return dataset, labels 

In [3]:
def splitDataset(dataset, axis, value):
    result = []
    
    for featureVector in dataset:
        if featureVector[axis] == value:
            reducedFeatureVector = featureVector[:axis]
            reducedFeatureVector.extend(featureVector[axis+1:])
            
            result.append(reducedFeatureVector)
    
    return result

In [15]:
def chooseBestFeatureToSplit(dataset):
    numFeatures = len(dataset[0]) - 1                                   # Last column are the labels
    
    baseEntropy = calcShannonEntropy(dataset)                           # Calculates entropy of entire dataset
    
    bestInfoGain = 0.0                                                  # Starts with 0                       
    bestFeature = -1                                                    # No best feature selected
    
    for feature in range(numFeatures):                              
        featureValuesList = [example[feature] for example in dataset]   # Gets of values of the X feature
        uniqueFeatureValuesList = set(featureValuesList)                # Removes the repeated ones
        
        newEntropy = 0.0
        
        for featureValue in uniqueFeatureValuesList:                    # Iterate over all unique values
            subDataset = splitDataset(dataset, feature, featureValue)   # Split the dataset from feature-value
            
            probability = len(subDataset) / float(len(dataset))         # Calculates probability for that feature-value
            newEntropy += probability + calcShannonEntropy(subDataset)  # Calculates entropy of the feature
            
        infoGain = baseEntropy - newEntropy
            
        print("Feature ", feature, ": ", newEntropy, " Gain:", infoGain)
        
        if infoGain > bestInfoGain:
            bestInfoGain = infoGain
            bestFeature = feature
    
    print("Dataset Entropy: ", baseEntropy, ", Best Entropy: ", bestInfoGain, " (", bestFeature, ")")
    return bestFeature

In [5]:
def majorityCount(classList):
    classCount = {}
    
    for item in classList:  
        if item not in classCount.keys():
            classCount[item] = 0
        
        classCount[item] += 1
        
    sortedClassCount = sorted(classCount.iteritems(), key=operator.itemgetter(1), reverse=True)
    
    return sortedClassCount[0][0]

In [17]:
def createTree(dataset, labels):
    classList = [example[-1] for example in dataset]
    
    if len(set(classList)) == 1:
        return classList[0]
    
    if len(dataset[0]) == 1:
        return majorityCount(classList)
    
    bestFeature = chooseBestFeatureToSplit(dataset)
    print("Dataset:", dataset, "\nBest Feature:", bestFeature)
    
    bestFeatureLabel = labels[bestFeature]
    
    myTree = { bestFeatureLabel: {}}
    
    del(labels[bestFeature])
    
    featureValues = [example[bestFeature] for example in dataset]
    uniqueFeatureValues = set(featureValues)
    
    for value in uniqueFeatureValues:
        subLabels = labels[:]
        
        myTree[bestFeatureLabel][value] = createTree(splitDataset(dataset, bestFeature, value), subLabels)
        
    return myTree

In [18]:
myDataset, labels = createDataset()

In [16]:
chooseBestFeatureToSplit(myDataset)

Feature  0 :  1.9182958340544896  Gain: -0.947345239599821
Feature  1 :  2.0  Gain: -1.0290494055453314
Dataset Entropy:  0.9709505944546686 , Best Entropy:  0.0  ( -1 )


-1

In [19]:
createTree(myDataset, labels)

# REPASAR createTree y chooseBestFeatureToSplit

Feature  0 :  1.9182958340544896  Gain: -0.947345239599821
Feature  1 :  2.0  Gain: -1.0290494055453314
Dataset Entropy:  0.9709505944546686 , Best Entropy:  0.0  ( -1 )
Dataset: [[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']] 
Best Feature: -1


{'flippers': {'no': 'no', 'yes': 'yes'}}

In [18]:
myTree

{'flippers': {0: 'no', 1: {'no surfacing': {'no': 'no', 'yes': 'yes'}}}}